In [ ]:
import pandas as pd

learned_indexes = ['alex', 'lipp', 'dili', 'dytis']
traditional_indexes = ['art', 'btree']

df = pd.read_csv('./single_thread_thp.csv')

# trim read_ratio = 0
df = df[df['read_ratio'] != 0]

grouped = df.groupby(['index_type', 'key_path', 'test_suite', 'init_table_size'])['throughput'].mean().reset_index()
learned_dfs = {index: grouped[grouped['index_type'] == index] for index in learned_indexes}
traditional_dfs = {index: grouped[grouped['index_type'] == index] for index in traditional_indexes}

# Counting the number of cases where the throughput of learned indexes is worse and better than traditional indexes
worse_cases_counts = {}
better_cases_counts = {}
for learned_index, learned_df in learned_dfs.items():
    worse_cases_counts[learned_index] = {}
    better_cases_counts[learned_index] = {}
    for traditional_index, traditional_df in traditional_dfs.items():
        # Merging the DataFrames
        merged_df = pd.merge(learned_df, traditional_df, on=['key_path', 'test_suite', 'init_table_size'], suffixes=(f'_{learned_index}', f'_{traditional_index}'))
        # Counting the worse and better cases
        worse_cases_count = (merged_df[f'throughput_{learned_index}'] < merged_df[f'throughput_{traditional_index}']).sum()
        better_cases_count = (merged_df[f'throughput_{learned_index}'] >= merged_df[f'throughput_{traditional_index}']).sum()
        worse_cases_counts[learned_index][traditional_index] = worse_cases_count
        better_cases_counts[learned_index][traditional_index] = better_cases_count

worse_cases_df = pd.DataFrame(worse_cases_counts)
worse_cases_df = worse_cases_df.T
print("learned indexes worse than traditional indexes cases:")
print(worse_cases_df)

better_cases_df = pd.DataFrame(better_cases_counts)
better_cases_df = better_cases_df.T
print("learned indexes better than traditional indexes cases:")
print(better_cases_df)

In [ ]:
# Counting the number of cases where the throughput of learned indexes is worse than traditional indexes for each dataset
worse_cases_by_key_path = {}
for key_path in grouped['key_path'].unique():
    worse_cases_by_key_path[key_path] = {}
    
    # Filtering data for the current key_path
    for learned_index, learned_df in learned_dfs.items():
        worse_cases_by_key_path[key_path][learned_index] = {}
        learned_key_path_df = learned_df[learned_df['key_path'] == key_path]
        
        for traditional_index, traditional_df in traditional_dfs.items():
            traditional_key_path_df = traditional_df[traditional_df['key_path'] == key_path]
            
            # Merging the DataFrames for the current key_path
            merged_df = pd.merge(
                learned_key_path_df,
                traditional_key_path_df,
                on=['key_path', 'test_suite', 'init_table_size'],
                suffixes=(f'_{learned_index}', f'_{traditional_index}')
            )
            
            # Counting the worse cases
            worse_cases_count = (merged_df[f'throughput_{learned_index}'] < merged_df[f'throughput_{traditional_index}']).sum()
            worse_cases_by_key_path[key_path][learned_index][traditional_index] = worse_cases_count

worse_cases_by_key_path_df = pd.DataFrame(worse_cases_by_key_path)
print("learned indexes worse than traditional indexes cases on each dataset:")
print(worse_cases_by_key_path_df)

In [ ]:
df_art = grouped[grouped['index_type'] == 'art']
art_worst_thpt_dict = {}
art_best_thpt_dict = {}
for i, row in df_art.iterrows():
    if row['key_path'] not in art_worst_thpt_dict:
        art_worst_thpt_dict[row['key_path']] = 1e9
    if row['key_path'] not in art_best_thpt_dict:
        art_best_thpt_dict[row['key_path']] = 0
    art_worst_thpt_dict[row['key_path']] = min(art_worst_thpt_dict[row['key_path']], row['throughput'])
    art_best_thpt_dict[row['key_path']] = max(art_best_thpt_dict[row['key_path']], row['throughput'])
# print("art worst:")
# print(art_worst_thpt_dict)
# print("art best:")
# print(art_best_thpt_dict)
# print("====================================")

df_alex = grouped[grouped['index_type'] == 'alex']
alex_worst_thpt_dict = {}
alex_best_thpt_dict = {}
for i, row in df_alex.iterrows():
    if row['key_path'] not in alex_worst_thpt_dict:
        alex_worst_thpt_dict[row['key_path']] = 1e9
    if row['key_path'] not in alex_best_thpt_dict:
        alex_best_thpt_dict[row['key_path']] = 0
    alex_worst_thpt_dict[row['key_path']] = min(alex_worst_thpt_dict[row['key_path']], row['throughput'])
    alex_best_thpt_dict[row['key_path']] = max(alex_best_thpt_dict[row['key_path']], row['throughput'])
# print("alex worst:")
# print(alex_worst_thpt_dict)
# print("alex best:")
# print(alex_best_thpt_dict)
# print("====================================")

df_lipp = grouped[grouped['index_type'] == 'lipp']
lipp_worst_thpt_dict = {}
lipp_best_thpt_dict = {}
for i, row in df_lipp.iterrows():
    if row['key_path'] not in lipp_worst_thpt_dict:
        lipp_worst_thpt_dict[row['key_path']] = 1e9
    if row['key_path'] not in lipp_best_thpt_dict:
        lipp_best_thpt_dict[row['key_path']] = 0
    lipp_worst_thpt_dict[row['key_path']] = min(lipp_worst_thpt_dict[row['key_path']], row['throughput'])
    lipp_best_thpt_dict[row['key_path']] = max(lipp_best_thpt_dict[row['key_path']], row['throughput'])
# print("lipp worst:")
# print(lipp_worst_thpt_dict)
# print("lipp best:")
# print(lipp_best_thpt_dict)
# print("====================================")

for key_path in art_worst_thpt_dict:
    print(f"key_path: {key_path}")
    print(f"alex worst: {(art_worst_thpt_dict[key_path] - alex_worst_thpt_dict[key_path]) / art_worst_thpt_dict[key_path]}")
    print(f"lipp worst: {(art_worst_thpt_dict[key_path] - lipp_worst_thpt_dict[key_path]) / art_worst_thpt_dict[key_path]}")
